**Импорт библиотек**

In [51]:
import os
import shutil
import json
from PreapreDataLib import create_folder_if_not_exists

from pycocotools.coco import COCO
from mmengine.config import *
from mmengine.runner import set_random_seed
from mmdet.datasets import CocoDataset
from mmdet.apis import init_detector, inference_detector
from mmengine.registry import *
import torch

In [5]:
DATASET_DIR = os.path.abspath('./DatasetCocoFormat/')
TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
TEST_DIR  = os.path.join(DATASET_DIR, 'test')
VAL_DIR = os.path.join(DATASET_DIR, 'val')
ANN_DIR = os.path.join(DATASET_DIR, 'annotations')
TRAIN_ANN_PTH = os.path.join(ANN_DIR, "train.json")
VAL_ANN_PTH = os.path.join(ANN_DIR, "val.json")
TEST_ANN_PTH = os.path.join(ANN_DIR, "test.json")

**Загрузка конфига Mask R-CNN ResNet101 FPN 1x**

In [56]:
CONFIGS_DIR = os.path.abspath("./configs")
create_folder_if_not_exists("./configs")
MRCNN_BASE_CONFIG_PTH = os.path.join(CONFIGS_DIR, "mask-rcnn_r101_fpn_1x_coco.py")
MRCNN_USER_CONFIG_PTH = os.path.join(CONFIGS_DIR, "mask-rcnn_r101_fpn_1x_coco_for_road_damages.py")
if not os.path.exists(MRCNN_BASE_CONFIG_PTH):
    !mim download mmdet --config mask-rcnn_r101_fpn_1x_coco --dest "./configs"

**Изменение конфига под задачу и датасет**

In [58]:
cfg = Config.fromfile(MRCNN_BASE_CONFIG_PTH)

cfg["model"]["roi_head"]["bbox_head"]["num_classes"] = 3
cfg["model"]["roi_head"]["mask_head"]["num_classes"] = 3

cfg["data_root"] = DATASET_DIR

cfg["train_dataloader"]["batch_size"] = 1
cfg["val_dataloader"]["batch_size"] = 1
cfg["test_dataloader"]["batch_size"] = 1

cfg["train_dataloader"]["num_workers"] = 1
cfg["val_dataloader"]["num_workers"] = 1
cfg["test_dataloader"]["num_workers"] = 1

cfg["train_dataloader"]["dataset"]["data_root"] = DATASET_DIR
cfg["val_dataloader"]["dataset"]["data_root"] = DATASET_DIR
cfg["test_dataloader"]["dataset"]["data_root"] = DATASET_DIR

cfg["train_dataloader"]["dataset"]["data_prefix"]["img"] = "train/"
cfg["val_dataloader"]["dataset"]["data_prefix"]["img"] = "val/"
cfg["test_dataloader"]["dataset"]["data_prefix"]["img"] = "test/"

cfg["train_dataloader"]["dataset"]["ann_file"] = TRAIN_ANN_PTH
cfg["val_dataloader"]["dataset"]["ann_file"] = VAL_ANN_PTH
cfg["test_dataloader"]["dataset"]["ann_file"] = TEST_ANN_PTH

cfg["val_evaluator"]["ann_file"] = VAL_ANN_PTH
cfg["test_evaluator"]["ann_file"] = TEST_ANN_PTH

cfg["test_pipeline"][1]["scale"] = (1280, 640)
cfg["train_pipeline"][2]["scale"] = (1280, 640)

cfg["train_dataloader"]["dataset"]["pipeline"][2]["scale"] = (1280, 640)
cfg["val_dataloader"]["dataset"]["pipeline"][1]["scale"] = (1280, 640)
cfg["test_dataloader"]["dataset"]["pipeline"][1]["scale"] = (1280, 640)

if os.path.exists(MRCNN_USER_CONFIG_PTH):
    os.remove(MRCNN_USER_CONFIG_PTH)
cfg.dump(MRCNN_USER_CONFIG_PTH)

user_config = Config.fromfile(MRCNN_USER_CONFIG_PTH)
print(user_config.pretty_text)

model = dict(
    type='MaskRCNN',
    data_preprocessor=dict(
        type='DetDataPreprocessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_mask=True,
        pad_size_divisor=32),
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained',
                      checkpoint='torchvision://resnet101')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        b